In [1]:
import numpy as np
import json


In [ ]:
problem = "toy"
file = "./instances/" + problem + ".json"

In [6]:
with open("./instances/small.json", 'r') as f:
    data = json.load(f)
with open("./instances/small_f.json", 'w') as f:
    json.dump(data, f, indent=4)


In [11]:


def gen_output(substations,  substation_substation_cables, turbines, sol_file):
    d = {
        "substations":substations,
        "substation_substation_cables":substation_substation_cables,
        "turbines":turbines
    }
    
    with open(sol_file, "w") as f:
        json.dump(d, f, indent=4)
    



In [12]:
def cost_under_scenario(scenario, data):
    return

In [18]:
def solver_naive(infile, outfile):
    with open(infile, 'r') as f:
        data = json.load(f)
        
    land_sub_c_types = data["land_substation_cable_types"]
    sub_sub_c_types = data["substation_substation_cable_types"]


    wind_turbines = data["wind_turbines"]
    wind_scenarios  =data["wind_scenarios"]
    sub_locs = data["substation_locations"]
    sub_types = data["substation_types"]

    fixed_cost_cable = data["general_parameters"]["fixed_cost_cable"]
    variable_cost_cable = data["general_parameters"]["variable_cost_cable"]
    curtailing_penalty = data["general_parameters"]["curtailing_penalty"]
    curtailing_cost = data["general_parameters"]["curtailing_cost"]
    main_land_station = data["general_parameters"]["main_land_station"]
    maximum_power = data["general_parameters"]["maximum_power"]
    maximum_curtailing = data["general_parameters"]["maximum_curtailing"]
    
    land_x = main_land_station["x"]
    land_y = main_land_station["y"]
    
    substations = []
    turbines = []
    sub_sub_c = []
    
    COP = maximum_power - maximum_curtailing
    
    max_sub_rating = max([r["rating"] for r in sub_types])
    used_sub_loc = []
    
    subs_cap = [0 for _ in range(len(sub_locs))]
    
    # Choose connections turbines - sub
    for wt in wind_turbines:
        id_t = wt["id"]
        x_t = wt["x"]
        y_t = wt["y"]
        
        
        min_dist = np.Infinity
        id_loc_real = 0
        idx_loc = 0
        for sub_idx, sub_loc in enumerate(sub_locs):
            if subs_cap[sub_idx] + COP > max_sub_rating:
                continue
            
            sub_x = sub_loc["x"]
            sub_y = sub_loc["y"]
            
            dist = np.sqrt(np.square(sub_x - x_t) + np.square(sub_y - y_t))
            dist+= np.sqrt(np.square(sub_x - land_x) + np.square(sub_y - land_y))
            
            if dist < min_dist:
                min_dist = dist
                id_loc_real = sub_loc["id"]
                idx_loc=sub_idx
        assert min_dist != np.Infinity, "No turbine - sub choice found. Must accept failure"
        
        subs_cap[idx_loc]+=COP
        if idx_loc not in used_sub_loc:
            used_sub_loc.append(idx_loc)
        turbines.append({
            "id": id_t,
            "substation_id": id_loc_real
        })
        
    # sub def
    for sub_idx, sub_loc in enumerate(sub_locs):
        sub_cap = subs_cap[sub_idx]
        if sub_cap == 0:
            continue
        sub_id = sub_loc["id"]
        
        # sub type choice
        min_cost = np.Infinity
        sub_type_idx = 0
        for idx, sub_type in enumerate(sub_types):
            if sub_type["rating"] >= sub_cap and sub_type["cost"] < min_cost:
                min_cost = sub_type["cost"]
                sub_type_idx = idx
        assert min_cost != np.Infinity, "No sub type choice found. Must accept failure"
        sub_type_id = sub_types[sub_type_idx]["id"]
        
        # land cable choice
        sub = sub_types[sub_type_idx]
        sub_x = sub_loc["x"]
        sub_y = sub_loc["y"]
        min_cost = np.Infinity
        c_idx = 0
        for idx, land_sub_c in enumerate(land_sub_c_types):
            if land_sub_c["rating"] < sub_cap:
                continue
            c_cost = land_sub_c["fixed_cost"]
            c_cost+= land_sub_c["variable_cost"] * np.sqrt(np.square(sub_x - land_x) + np.square(sub_y - land_y))
            
            if c_cost < min_cost:
                min_cost = c_cost
                c_idx = idx
        c_id = land_sub_c_types[c_idx]["id"]
        
        substations.append({
            "id": sub_id,
            "land_cable_type": c_id,
            "substation_type": sub_type_id
        })
        
    # sub to sub cables
    
    gen_output(substations, sub_sub_c, turbines, outfile)
    

In [19]:
solver_naive("./instances/toy.json", "./sols/toy_sol.json")
solver_naive("./instances/small.json", "./sols/small_sol.json")
solver_naive("./instances/medium.json", "./sols/medium_sol.json")
solver_naive("./instances/large.json", "./sols/large_sol.json")
solver_naive("./instances/huge.json", "./sols/huge_sol.json")